In [2]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast
import nltk
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import gzip
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
from time import sleep
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.svm import SVC
from sklearn import svm

In [7]:
path = "Downloads/assignment1/train_Category.json"

In [8]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield ast.literal_eval(l)
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [9]:
data = list(parseDataFromFile(path))

In [5]:
data[0]

{'n_votes': 0,
 'review_id': 'r99763621',
 'user_id': 'u17334941',
 'review_text': "Genuinely enthralling. If Collins or Bernard did invent this out of whole cloth, they deserve a medal for imagination. Lets leave the veracity aside for a moment - always a touchy subject when it comes to real life stories of the occult - and talk about the contents. \n The Black Alchemist covers a period of two years in which Collins, a magician, and Bernard, a psychic, undertook a series of psychic quests that put them in opposition with the titular Black Alchemist. As entertainment goes, the combination of harrowing discoveries, ancient lore, and going down the pub for a cigarette and a Guinness, trying to make sense of it all while a hen party screams at each other, is a winner. It is simultaneously down to earth and out of this world. \n It reads fast, both because of the curiousity and because Collins has a very clear writing style. Sometimes its a little clunky or over repetitive and there's a fe

In [6]:
X_train_raw = data[:190000]
y_train_raw = data[:190000]
X_valid_raw = data[190000:]
y_valid_raw = data[190000:]

In [7]:
all_reviews = [d['review_text'] for d in tqdm(data)]

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tqdm(all_reviews))
y = [d['genreID'] for d in tqdm(data)]
y_train = [d['genreID'] for d in tqdm(y_train_raw)]
y_valid = [d['genreID'] for d in tqdm(y_valid_raw)]
pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))

In [9]:
mod = LogisticRegression(C=1)
X_train = X[:190000]

In [10]:
mod.fit(X_train, y_train)

/Users/zawinglee/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zawinglee/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
X_valid = X[190000:]
pred_valid = mod.predict(X_valid)
accuracy_score(y_valid, pred_valid)

0.7785

In [12]:
testing = list(readGz("Downloads/assignment1/test_Category.json.gz"))

In [13]:
test_review = [d['review_text'] for d in tqdm(testing)]

In [14]:
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(test_review))

In [15]:
pred = mod.predict(tfidf)

In [16]:
index = 0
predictions = open("Downloads/assignment1/predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for l in readGz("Downloads/assignment1/test_Category.json.gz"):
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(pred[index]) + "\n")
    index += 1
predictions.close()

SVM

In [17]:
clf = svm.LinearSVC(random_state=0, tol=1e-5)

In [18]:
clf.fit(X_train, y_train) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)

In [19]:
pred_valid = clf.predict(X_valid)
accuracy_score(y_valid, pred_valid)

0.7935

In [28]:
pred_svc = clf.predict(tfidf)

In [29]:
index = 0
predictions = open("Downloads/assignment1/predictions_Category_svc.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for l in readGz("Downloads/assignment1/test_Category.json.gz"):
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(pred_svc[index]) + "\n")
    index += 1
predictions.close()

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier()

In [34]:
clf2.fit(X_train, y_train) 

/Users/zawinglee/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
pred_valid = clf2.predict(X_valid)
accuracy_score(y_valid, pred_valid)

0.5121

In [36]:
pred_ft = clf.predict(tfidf)

In [37]:
index = 0
predictions = open("Downloads/assignment1/predictions_Category_svc.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for l in readGz("Downloads/assignment1/test_Category.json.gz"):
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(pred_ft[index]) + "\n")
    index += 1
predictions.close()